In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from unidecode import unidecode

import os
from PIL import Image, ImageOps
import numpy as np
from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
import keras.backend as K
from keras.callbacks import ModelCheckpoint

In [2]:
def captcha_solver(address):
    filenames = []
    for dirname, _, files in os.walk(address):
        for f in files:
            filenames = np.append(filenames, f)

    num_samples = len(filenames)
    print('number of samples: ', num_samples)

    char_list = "0123456789"


    def encode_to_labels(txt):
        # encoding each label into list of digits
        encoded_list = []
        for char in txt:
            encoded_list.append(char_list.index(char))

        return encoded_list


    path = r'D:/hiweb/exports/temp/'

    # lists for training dataset
    training_img = []  # the images for training the model
    training_txt = []  # the labels
    train_input_length = []  # the input of LSTM part of the model
    train_label_length = []  # the label's length (4 to 7)
    train_orig_txt = []

    # lists for validation dataset
    valid_img = []
    valid_txt = []
    valid_input_length = []
    valid_label_length = []
    valid_orig_txt = []

    max_label_len = 0  # max length for our labels (in this case 7)

    for file in filenames:
        raw = Image.open(path + file)
        gray = ImageOps.grayscale(raw)
        img = np.array(gray)
        img = np.expand_dims(img, axis=2)
        img = img / 255.

        txt = file.split('.')[0]

        if len(txt) > max_label_len:
            max_label_len = len(txt)

        # split the dataset (85% train, 15% test)
        if np.random.rand() >= 0:
            valid_orig_txt.append(txt)
            valid_label_length.append(len(txt))
            valid_input_length.append(75)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(txt))
        else:
            train_orig_txt.append(txt)
            train_label_length.append(len(txt))
            train_input_length.append(75)
            training_img.append(img)
            training_txt.append(encode_to_labels(txt))

    train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value=len(char_list))
    valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value=len(char_list))


    def ctc_lambda_func(args):
        y_pred, labels, input_length, label_length = args

        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


    training_img = np.array(training_img)
    train_input_length = np.array(train_input_length)
    train_label_length = np.array(train_label_length)

    valid_img = np.array(valid_img)
    valid_input_length = np.array(valid_input_length)
    valid_label_length = np.array(valid_label_length)

    training_txt = np.array(training_txt)
    valid_txt = np.array(valid_txt)

    inputs = Input(shape=(64, 306, 1))

    conv_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

    conv_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

    conv_3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool_2)
    conv_4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_3)
    pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

    conv_5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool_4)
    batch_norm_5 = BatchNormalization()(conv_5)

    conv_6 = Conv2D(512, (3, 3), activation='relu', padding='same')(batch_norm_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

    conv_7 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool_6)
    batch_norm_7 = BatchNormalization()(conv_7)
    pool_7 = MaxPool2D(pool_size=(2, 1))(batch_norm_7)

    conv_8 = Conv2D(512, (2, 2), activation='relu')(pool_7)

    squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)

    blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(squeezed)
    blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(blstm_1)

    outputs = Dense(len(char_list) + 1, activation='softmax')(blstm_2)
    prediction_model = Model(inputs, outputs)
    # load the model weights
    prediction_model.load_weights('D:\hiweb\model_weights_V1_1413data.hdf5')

    # predict outputs on validation images
    prediction = prediction_model.predict(valid_img[:20])

    # use CTC decoder
    out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                                   greedy=True)[0][0])

    # see the results
    i = 0
    pred = ''
    for x in out:
    #     print("original_text =  ", valid_orig_txt[i])
    #     print("predicted text = ", end='')
        for p in x:
            if int(p) != -1:

                pred += str(p)
    #             print(char_list[int(p)], end='')
        print('\n')
        i += 1
    return((pred))

In [40]:
URL = 'https://adsl.tci.ir/'
LOGIN_ROUTE = 'panel/login/'

# HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 'origin': URL, 'referer': URL + LOGIN_ROUTE}

start_with_zero = True
while start_with_zero:

    s = requests.session()
    page = s.get(URL)
    soup = BeautifulSoup(page.text, 'lxml')

    image_url = soup.select('#loginCaptchaImage')
    img_data = [img['src'] for img in image_url]
    r = img_data[0].split('=')
    print(r[1])
    with open(f'D:/hiweb/exports/temp/0000.jpg', 'wb') as handle:
        response = requests.get(img_data[0], stream=True)

        if not response.ok:
            print(response)

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)
    captcha = captcha_solver('D:/hiweb/exports/temp')
    for i in range (len(captcha) - 1):
        if captcha[i] != '0':
            start_with_zero = False
            break

            
PHPSESSID = s.cookies['PHPSESSID']
cookiesession1 = s.cookies['cookiesession1']
print(f'captcha = {captcha}')

print(f'PHPSESSID = {PHPSESSID} \ncookiesession1 = {cookiesession1} \nr1 = {r[1]}\nurl = {URL + LOGIN_ROUTE + r[1]}')


1675058130
number of samples:  1
1/1 [==============================] - 1s 785ms/step


captcha = 128094
PHPSESSID = 6j9svec064holqi0e2s4h2nr4f 
cookiesession1 = 678B2868C1DFEF1B4B16E1C82A550FFC 
r1 = 1675058130
url = https://adsl.tci.ir/panel/login/1675058130


In [41]:
cookies = {
    'cookiesession1': cookiesession1,
    'PHPSESSID': PHPSESSID,
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,fa;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'cookiesession1=678B28680532EBC101335147AF24C6F9; PHPSESSID=289cb3gmevoh3v21gd0sp9cn15',
    'Origin': 'https://adsl.tci.ir',
    'Referer': 'https://adsl.tci.ir/panel/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}



data = {
    'redirect': '',
    'username': '2177492386',
    'password': '216011',
    'captcha': captcha,
    'LoginFromWeb': '',
}

response = s.post(URL + LOGIN_ROUTE + r[1], cookies=cookies, headers=headers, data=data)

In [43]:
response

<Response [200]>

In [42]:

soup = BeautifulSoup(s.get(URL + 'panel/').text, 'lxml')
print(soup)

<!DOCTYPE html>
<html dir="rtl" lang="fa">
<head>
<title>ورود به سامانه — سامانه مدیریت مشتریان اینترنت شرکت مخابرات ایران</title>
<meta content="#1E87F0" name="theme-color"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="Partak Group Ltd." name="Development"/>
<meta content="PartakCRM" name="description"/>
<meta content="پژواک اندیشه اسپادانا (پارتاک)" name="author"/>
<link href="https://adsl.tci.ir/panel/favicon.png" rel="icon" type="image/png"/>
<link href="https://adsl.tci.ir/panel/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<link href="https://adsl.tci.ir/panel/assets/css/fonts.css" rel="stylesheet" type="text/css"/>
<link href="https://adsl.tci.ir/panel/assets/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<link href="https://adsl.tci.ir/panel/assets/css/uikit/uikit-rtl.min.css" rel="stylesheet" type="text/css"/>
<link href="

In [4]:
# captcha = captcha_solver('D:/hiweb/exports/temp')
# print((captcha))

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 'origin': URL, 'referer': URL + LOGIN_ROUTE + r[1]}


login_payload = {
#     'redirect': '',
    'username': 2177492386,
    'password': 216011,
    'captcha': int(captcha),
#     'LoginFromWeb': '',
}


login_req = s.post(URL + LOGIN_ROUTE + r[1], headers=HEADERS, data=login_payload)
print(login_req.status_code)



200


In [5]:
URL + LOGIN_ROUTE + r[1]

'https://adsl.tci.ir/panel/login/1675054648'

In [6]:
cookies = login_req.cookies
soup = BeautifulSoup(s.get(URL + 'panel/change-service').text, 'lxml')
services = soup.find('div', id='service-list')

headers = ['service name', 'traffic', 'price']
service_name, traffic, price = [], [], []

tci_services = pd.DataFrame(columns=headers)

In [7]:
soup

<!DOCTYPE html>
<html dir="rtl" lang="fa">
<head>
<title>ورود به سامانه — سامانه مدیریت مشتریان اینترنت شرکت مخابرات ایران</title>
<meta content="#1E87F0" name="theme-color"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="Partak Group Ltd." name="Development"/>
<meta content="PartakCRM" name="description"/>
<meta content="پژواک اندیشه اسپادانا (پارتاک)" name="author"/>
<link href="https://adsl.tci.ir/panel/favicon.png" rel="icon" type="image/png"/>
<link href="https://adsl.tci.ir/panel/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<link href="https://adsl.tci.ir/panel/assets/css/fonts.css" rel="stylesheet" type="text/css"/>
<link href="https://adsl.tci.ir/panel/assets/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<link href="https://adsl.tci.ir/panel/assets/css/uikit/uikit-rtl.min.css" rel="stylesheet" type="text/css"/>
<link href="

In [ ]:
for item in services.find_all('div','service-list'):
    title = (item.find_all('p', 'uk-text-bold'))
    traffic_temp = (item.find_all('h3', 'uk-card-title'))
    price_temp = (item.find_all('h3', 'uk-margin-remove-bottom'))
    service_name.append(title)
    traffic.append(traffic_temp)
    price.append((price_temp))

In [16]:
response = requests.get(URL + LOGIN_ROUTE + r[1])
response.cookies

<RequestsCookieJar[Cookie(version=0, name='PHPSESSID', value='1hjm6lur8014pd2u4lmrim8lo0', port=None, port_specified=False, domain='adsl.tci.ir', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='cookiesession1', value='678B2868973B0813565AE404C92899BD', port=None, port_specified=False, domain='adsl.tci.ir', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1706593242, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [18]:
response.cookies['PHPSESSID']

'1hjm6lur8014pd2u4lmrim8lo0'

In [19]:
response.cookies['cookiesession1']

'678B2868973B0813565AE404C92899BD'